In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup, Tag, Comment

In [3]:
import random
import time
import re

### Retrieve and save HTML

In [4]:
url = "https://www.berlin.de/rbmskzl/"

In [5]:
response = requests.get(url)

In [6]:
response.status_code

200

In [7]:
html_text = response.text

In [8]:
html_text[:100]

'<!doctype html>\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="de" lang="de">\n<head prefix="og'

In [9]:
with open('berlin_rbmskzl_main_page.html', 'w') as openfile:
    openfile.write(html_text)

### Get text of the HTML 
(same as in the [brandenburg notebook](https://github.com/dh-network/quadriga-fs-2/blob/develop/html_parsing/brandenburg-landesportal.ipynb))

In [10]:
with open('berlin_rbmskzl_main_page.html') as openfile:
    html_text = openfile.read()

In [11]:
soup = BeautifulSoup(html_text)

In [12]:
# https://stackoverflow.com/questions/1936466/how-to-scrape-only-visible-webpage-text-with-beautifulsoup

# Only get text that is visible on the website 
# Inlcudes ads, pointers to more info and so on 

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(soup):
    texts = soup.findAll(string=True)
    visible_texts = filter(tag_visible, texts)  
    return [t.strip() for t in visible_texts if len(t) > 1]

In [13]:
text = text_from_html(soup)

In [14]:
text

['Der Regierende Bürgermeister',
 'Senatskanzlei',
 'Informationen und Links zur Barrierefreiheit öffnen',
 'Barrierefrei',
 'Barrierefreiheit',
 'Schließen: Barrierefreiheit',
 'Leichte Sprache',
 'DGS',
 'Wie barrierefrei ist diese Webseite?',
 'Erklärung zur Barrierefreiheit',
 'Haben Sie Anmerkungen oder Fragen zur Barrierefreiheit dieser Webseite?',
 'Kontakt zur Ansprechperson',
 'Wo gibt es zusätzliche Informationen zur Barrierefreiheit im Land Berlin?',
 'Landesbeauftragte für digitale Barrierefreiheit',
 'Suche',
 'Suche',
 'Suche schließen',
 'Suche auf der Internetseite',
 'Suchen',
 'Menü',
 'Hauptnavigation',
 'Menü schließen',
 'Aktuelles',
 'Termine',
 'Abonnement',
 'Presse\xadmitteilungen',
 '2023',
 '2022',
 '2021',
 'Suche und Abonnement',
 'Media',
 'Veranstaltungen',
 'Kai Wegner vor Ort',
 'Ausstellungen',
 'Rückblick',
 'Auszeichnungen und Ehrungen',
 'Ehren\xadbürgerschaft Berlins',
 'Ehren\xadgrabstätten',
 'Ernst-Reuter-Plakette',
 'Verdienstorden des Landes B

### Parse HTML (strukturierte parsen, one page)

#### 1. Parse the top news section of the website

![berlin_de_top](berlin_de_top.png)

If we inspect the unredlying HTML code, we'll see that it is in a `<div>` element with a CSS class `'herounit-homepage herounit-homepage--default'`. Let's retrieve this div using BeautifulSoup syntax:

In [15]:
topdiv = soup.find("div", {"class": "herounit-homepage herounit-homepage--default"})

In [16]:
type(topdiv)

bs4.element.Tag

In [17]:
print(topdiv.prettify())

<div class="herounit-homepage herounit-homepage--default">
 <h1 class="title">
  Der Regierende Bürgermeister von Berlin - Senatskanzlei
 </h1>
 <div class="modul-buehne buehne--tileslayout">
  <ul class="buhne__list--teaser">
   <li>
    <div class="modul-teaser_buehne" data-add-clickable-area="smart">
     <div class="teaser_buehne__left">
      <div class="image">
       <!-- Image.view -->
       <div class="image__image image__image" style="">
        <img alt="Kai Wegner" class="jpg" data-orig="/rbmskzl/aktuelles/veranstaltungen/kai-wegner-vor-ort/crop_1498.0833282470703_749.0416641235352_0.9166717529296875_157.39999389648438_1500_1000_e596139a8ed355df53171ce6e5df1a2a_att32055.jpg" loading="lazy" src="/imgscaler/cACcMGDdMVk7CupjfERExNV8smwSnaAyBTAGMV0L-Ac/rbig2zu1/L3N5czExLXByb2QvcmJtc2t6bC9ha3R1ZWxsZXMvdmVyYW5zdGFsdHVuZ2VuL2thaS13ZWduZXItdm9yLW9ydC9jcm9wXzE0OTguMDgzMzI4MjQ3MDcwM183NDkuMDQxNjY0MTIzNTM1Ml8wLjkxNjY3MTc1MjkyOTY4NzVfMTU3LjM5OTk5Mzg5NjQ4NDM4XzE1MDBfMTAwMF9lNTk2MTM5YTh

In [18]:
topdiv_h2titles = topdiv.find_all('h2')

In [19]:
topdiv_h2titles

[<h2 class="title">Kai Wegner vor Ort</h2>,
 <h2 class="title">MPK in Leipzig</h2>,
 <h2 class="title">Elektronische Wohnsitzanmeldung</h2>,
 <h2 class="title">Digitalministerkonferenz</h2>]

In [20]:
topdiv_texts =  topdiv.find_all('p', {"class":"text"})

In [21]:
topdiv_texts

[<p class="text">
                     Der Regierende Bürgermeister besucht am 20. November den Bezirk Spandau und stellt sich den Fragen der Berlinerinnen und Berliner. Melden Sie sich an!                        </p>,
 <p class="text">
                 Ein Schwerpunkte der Ministerpräsidentenkonferenz war die Migrationspolitik - Kai Wegner äußert sich zu den Beschlüssen.                        </p>,
 <p class="text">
                 Ab sofort können An- und Ummeldungen von Wohnsitzen bequem online erledigt werden.                        </p>,
 <p class="text">
                 Digitalisierung in Deutschland zügiger vorantreiben und digitale Transformation gestalten - das ist das Ziel der Treffen der Digitalverantwortlichen der Länder.                        </p>]

In [22]:
topdiv_texts = [x.text.strip() for x in topdiv_texts]

In [23]:
topdiv_texts

['Der Regierende Bürgermeister besucht am 20. November den Bezirk Spandau und stellt sich den Fragen der Berlinerinnen und Berliner. Melden Sie sich an!',
 'Ein Schwerpunkte der Ministerpräsidentenkonferenz war die Migrationspolitik - Kai Wegner äußert sich zu den Beschlüssen.',
 'Ab sofort können An- und Ummeldungen von Wohnsitzen bequem online erledigt werden.',
 'Digitalisierung in Deutschland zügiger vorantreiben und digitale Transformation gestalten - das ist das Ziel der Treffen der Digitalverantwortlichen der Länder.']

In [24]:
topdiv_h2titles = [x.text.strip() for x in topdiv_h2titles]

In [25]:
resulting_data = []

for i, text in enumerate(topdiv_texts):
    new_entry = {}
    new_entry['title'] = topdiv_h2titles[i]
    new_entry['text'] = text
    resulting_data.append(new_entry)

In [26]:
resulting_data

[{'title': 'Kai Wegner vor Ort',
  'text': 'Der Regierende Bürgermeister besucht am 20. November den Bezirk Spandau und stellt sich den Fragen der Berlinerinnen und Berliner. Melden Sie sich an!'},
 {'title': 'MPK in Leipzig',
  'text': 'Ein Schwerpunkte der Ministerpräsidentenkonferenz war die Migrationspolitik - Kai Wegner äußert sich zu den Beschlüssen.'},
 {'title': 'Elektronische Wohnsitzanmeldung',
  'text': 'Ab sofort können An- und Ummeldungen von Wohnsitzen bequem online erledigt werden.'},
 {'title': 'Digitalministerkonferenz',
  'text': 'Digitalisierung in Deutschland zügiger vorantreiben und digitale Transformation gestalten - das ist das Ziel der Treffen der Digitalverantwortlichen der Länder.'}]

### Parse HTML for links

In [27]:
links = soup.find_all('a', href=True)

In [28]:
links[1]

<a href="/rbmskzl" title='Startseite von "Der Regierende Bürgermeister Senatskanzlei"'><span class="institution">Der Regierende Bürgermeister</span><span class="title">Senatskanzlei</span></a>

#### Separate internal and external links

In [29]:
internal_links = []
external_links = []

for link in links:
    link_url = link['href']
    if link_url.startswith('/'):
        internal_links.append(link_url)
    else:
        external_links.append(link_url)                           

In [30]:
external_links[:5]

['https://www.berlin.de',
 'https://twitter.com/RegBerlin',
 'https://www.facebook.com/RegBerlin/',
 'https://www.instagram.com/regberlin/',
 'https://www.linkedin.com/company/regberlin/']

In [31]:
internal_links[:15]

['/rbmskzl',
 '/rbmskzl/leichte-sprache/',
 '/rbmskzl/gebaerdensprache/',
 '/rbmskzl/barrierefreiheitserklaerung.879589.php',
 '/rbmskzl/barrierefreiheitserklaerung.879589.php#contactbfe',
 '/rbmskzl/barrierefreiheitserklaerung.879589.php#contactbfelb',
 '/rbmskzl/aktuelles/',
 '/rbmskzl/aktuelles/termine/',
 '/rbmskzl/aktuelles/termine/abonnement/',
 '/rbmskzl/aktuelles/pressemitteilungen/',
 '/rbmskzl/aktuelles/pressemitteilungen/2023/',
 '/rbmskzl/aktuelles/pressemitteilungen/2022/',
 '/rbmskzl/aktuelles/pressemitteilungen/2021/',
 '/rbmskzl/aktuelles/pressemitteilungen/suche-und-abonnement/',
 '/rbmskzl/aktuelles/media/']

#### parsing all internal links and getting text

In [33]:
all_links_data = []

for link in internal_links:
    data = {}
    full_url = f'https://www.berlin.de{link}'
    response = requests.get(full_url)
    if response.status_code == 200:
        html_text = response.text
        soup = BeautifulSoup(html_text)
        text = text_from_html(soup)
        data['url'] = full_url
        data['text'] = text
        all_links_data.append(data)
    else:
        print(f'error {response.status_code}')
    

error 429
error 425
error 425


In [34]:
len(all_links_data)

169

In [35]:
all_links_data[:5]

[{'url': 'https://www.berlin.de/rbmskzl',
  'text': ['Der Regierende Bürgermeister',
   'Senatskanzlei',
   'Informationen und Links zur Barrierefreiheit öffnen',
   'Barrierefrei',
   'Barrierefreiheit',
   'Schließen: Barrierefreiheit',
   'Leichte Sprache',
   'DGS',
   'Wie barrierefrei ist diese Webseite?',
   'Erklärung zur Barrierefreiheit',
   'Haben Sie Anmerkungen oder Fragen zur Barrierefreiheit dieser Webseite?',
   'Kontakt zur Ansprechperson',
   'Wo gibt es zusätzliche Informationen zur Barrierefreiheit im Land Berlin?',
   'Landesbeauftragte für digitale Barrierefreiheit',
   'Suche',
   'Suche',
   'Suche schließen',
   'Suche auf der Internetseite',
   'Suchen',
   'Menü',
   'Hauptnavigation',
   'Menü schließen',
   'Aktuelles',
   'Termine',
   'Abonnement',
   'Presse\xadmitteilungen',
   '2023',
   '2022',
   '2021',
   'Suche und Abonnement',
   'Media',
   'Veranstaltungen',
   'Kai Wegner vor Ort',
   'Ausstellungen',
   'Rückblick',
   'Auszeichnungen und Ehr

### Save as JSON

In [36]:
from datetime import datetime
import json

In [37]:
date = datetime.now().strftime('%Y-%m-%d')

In [38]:
with open(f'{date}_berlin-senatskanzlei.json', 'w') as json_out:
    json.dump(all_links_data, json_out, ensure_ascii=False)

### Creating a diachronic corpus of press-releases -- ⚠️ Work in Progress!

Let us examine the page with press releases: https://www.berlin.de/presse/pressemitteilungen
Iа we look at the menu, we'll see that it contains the press-releases of the Berlin authorities since 1993... The links to the pages with press-releases are quite predictable:  

* https://www.berlin.de/presse/pressemitteilungen/index/index/page/2
* https://www.berlin.de/presse/pressemitteilungen/index/index/page/3
* https://www.berlin.de/presse/pressemitteilungen/index/index/page/4
* https://www.berlin.de/presse/pressemitteilungen/index/index/page/5
* ...
* ...
* https://www.berlin.de/presse/pressemitteilungen/index/index/page/7193
* https://www.berlin.de/presse/pressemitteilungen/index/index/page/7194
* https://www.berlin.de/presse/pressemitteilungen/index/index/page/7195


<s>So we could easily go over all of them and save the html files</s>  This does not work from python, apparently the server detects crawling

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

for i in range(1, 7196):
    url = f'https://www.berlin.de/presse/pressemitteilungen/index/index/page/{i}'
    response = requests.get(full_url, headers=headers)
    time.sleep(1+random.random())
    with open(f'html_files/{i}.html', 'w') as outfile:
        outfile.write(response.text)
    print(url)

switching to selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome()

In [ ]:
def parse_url_sel(url, links, filetowrite):
    driver.get(url)
    # Find all the anchor tags
    anchors = driver.find_elements(By.TAG_NAME, "a")

    # Extract and store the URLs from the href attribute
    for anchor in anchors:
        href = anchor.get_attribute("href")
        if href:  # Check if href is not None
            if 'pressemitteilung.' in href: # Check that the link looks like a press-release
                links.append(href)
                filetowrite.write(f'{href}\n')    

#### parsing for links to press-releases

In [ ]:
all_links = []

In [ ]:
from tqdm import tqdm

In [ ]:
with open('presslinks.txt', 'a') as linksfile:
    for i in tqdm(range(1215, 7196)):
        time.sleep(random.random())
        url = f'https://www.berlin.de/presse/pressemitteilungen/index/index/page/{i}'
        parse_url_sel(url, all_links, linksfile)

In [ ]:
with open('presslinks.txt') as presslinks:
    lines = presslinks.readlines()

In [ ]:
lines = [x.strip() for x in lines]

In [ ]:
lines[:3]

#### getting texts of press-releases

In [ ]:
#data = []

In [ ]:
for link in tqdm(lines):
    if link not in urls:
        this_release = {}
        #time.sleep(random.random())
        response = requests.get(link)
        if response.status_code == 200:
            html = response.text
            this_release['url'] = link
            this_release['html'] = html
            data.append(this_release)
        else:
            print(f'error {response.status_code}')

### Dump it to JSON!

In [ ]:
len(data)

In [ ]:
with open(f'all_press_releases.json', 'w') as json_out:
    json.dump(data, json_out, ensure_ascii=False)

In [ ]:
uniqie_urls = set()
unique_data = []
for i in data:
    thisurl = i['url']
    if thisurl not in uniqie_urls:
        unique_data.append(i)
        uniqie_urls.add(thisurl)

In [ ]:
len(unique_data)

In [ ]:
unique_data[1]

In [ ]:
del data

In [ ]:
ordered_texts = []
ordered_urls = []
for i in tqdm(unique_data):
    soup = BeautifulSoup(i['html'])
    pure_text = text_from_html(soup)
    ordered_texts.append(pure_text)
    ordered_urls.append(i['url'])
    #i['text'] = pure_text
    #del i['html']

In [ ]:
print('\n'.join(ordered_texts[1]))

In [ ]:
ordered_texts_str = ['\n'.join(x) for x in ordered_texts]

### process with pandas

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
tqdm.pandas()

In [ ]:
df = pd.DataFrame()

In [ ]:
df['url'] = pd.Series(ordered_urls)

In [ ]:
df['text'] = pd.Series(ordered_texts_str)

In [ ]:
df

In [ ]:
def get_date(whole_text):
    '''looks for date in raw text in format \d\d\.\d\d\.\d\d\d\d
    returns in w3c format if found
    '''
    datesearch = re.search(r'(\d\d)\.(\d\d)\.((199|200|201|202)\d)', whole_text) 
    if datesearch is not None:
        return f'{datesearch.group(3)}-{datesearch.group(2)}-{datesearch.group(1)}' 
    return None
    

In [ ]:
print(df['text'][1])

In [ ]:
get_date(df['text'][15])

In [ ]:
df['date'] = df['text'].apply(get_date)

In [ ]:
df = df.dropna()

In [ ]:
df

In [ ]:
df['month'] = df['date'].apply(lambda x: x[:-3])

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("de_core_news_sm")

In [ ]:
def lemmatize(some_text):
    doc = nlp(some_text)
    lemmas = [token.lemma_ for token in doc]
    return lemmas

In [ ]:
lemmatize('Haben Sie Anmerkungen oder Fragen zur Barrierefreiheit dieser Webseite?')

In [ ]:
df['textlem'] = df['text'].progress_apply(lemmatize)

In [ ]:
def count_word(some_lemmas, some_keyword):
    some_keyword = some_keyword.lower()
    counter = 0
    for lemma in some_lemmas:
        if lemma.lower() == some_keyword:
            counter+=1
    return counter        

In [ ]:
count_word(['aaa','aaa','aa', 'aa', 'aa', 'aa', 'a'], 'b')

In [ ]:
df['ukr_freq'] = df['textlem'].apply(count_word, some_keyword='Ukraine')

In [ ]:
df

In [ ]:
df

In [ ]:
df.groupby('month').sum()['virus_freq'].plot(figsize=(10,8), title='Virus')
plt.xticks(rotation=45);

In [ ]:
(df.groupby('month').sum()['virus_freq']/df.groupby('month').count()['url']).plot(figsize=(10,8), title='Virus')

In [ ]:
df.groupby('month').sum()['ukr_freq'].plot(figsize=(10,8), title='Ukraine')

In [ ]:
df.groupby('month').sum()['migrant_freq'].plot(figsize=(10,8), title='Migrant')
plt.xticks(rotation=45);

In [ ]:
df.to_json('berlin_parsed.json')

### Older stuff

In [ ]:
with open(f'all_press_releases.json', 'w') as json_out:
    json.dump(unique_data, json_out, ensure_ascii=False)

In [ ]:
json.dump(data)

In [ ]:
urls = set()
for i in data:
    urls.add(i['url'])

In [ ]:
len(urls)

In [ ]:
data[1]['url']

In [ ]:
soup = BeautifulSoup(data[1]['html'])

In [ ]:
text = text_from_html(soup)

In [ ]:
text

In [ ]:
test_list = []
with open('testfile.txt', 'w') as linksfile:
    parse_url_sel("https://www.berlin.de/presse/pressemitteilungen/index/index/page/2", test_list, linksfile)

In [ ]:
driver.get("https://www.berlin.de/presse/pressemitteilungen/index/index/page/2")

In [ ]:
# Find all the anchor tags
anchors = driver.find_elements(By.TAG_NAME, "a")

# Extract and store the URLs from the href attribute
links = []
for anchor in anchors:
    href = anchor.get_attribute("href")
    if href:  # Check if href is not None
        if 'pressemitteilung.' in href: # Check that the link looks like a press-release
            links.append(href)


In [ ]:
links

*To be continued*